In [1]:
import numpy as np
import math
import csv
from statistics import mode
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten, MaxPooling2D
from keras.optimizers import SGD

Using TensorFlow backend.


In [11]:
# SET50 (unrealize)
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 
         'CPF', 'CPN', 'DTAC', 'EGCO', 'GLOBAL', 'HMPRO', 
         'INTUCH', 'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 'PTT',
        'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

# remove (low data) : TPIPP WHA SPRC SAWAD PSH MTLS IVL GPSC EA CBG BPP BEAUTY PTTGC

In [3]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('./SET50/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        temp_value = 0
        for row in file_data:
            if row[2] is '':
                continue
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = (float(row[2]) + float(row[3]))/2.0
                if temp_value != 0:
                    unrealize = (temp - temp_value) / temp_value
                else:
                    unrealize = 0
                unrealize = ["{0:.3f}".format(unrealize)]
                stock_data.append(unrealize)
                temp_value = temp
                
    return stock_data

def mean_square_error(actual, expected):
    length = len(actual)
    sums_error = 0.0
    for i, j in zip(actual, expected):
        sums_error += (i[0] - j[0])**2
    
    return sums_error / length

In [17]:
stock_datas = [ load_stock_data(x, 2007, 2017) for x in SET50 ]

min_count = len(stock_datas[0])

for index, element in enumerate(stock_datas):
    if len(element) < min_count:
        min_count = len(element)
        
min_count

2043

In [28]:
model = Sequential()
    
model.add(Conv2D(64, (3, 3), input_shape=(len(stock_datas), 30, 1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3 ,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))   
      
model.add(Flatten())  
model.add(Dense(512))  
model.add(Activation('relu'))  
model.add(Dropout(0.5))
model.add(Dense(len(stock_datas)))# equal symbols

sgd = SGD(lr = 0.01, momentum = 0.9, decay= 1e-6, nesterov = True)

model.compile(loss='mean_squared_error', optimizer=sgd)

In [29]:
rounds = min_count - 60
train_index = int(rounds * 0.9)

In [30]:
data = []
count = 0
index = 0
while index < rounds:
    temp = [i[index : index + 30] for i in stock_datas]
    data.append(temp)
    index += 1
    
x = np.asarray(data)
x_train = x[0: train_index].astype('float32')
x_test = x[train_index: rounds].astype('float32')

print(x_train.shape, x_test.shape)

(1784, 37, 30, 1) (199, 37, 30, 1)


In [35]:
target = []
index = 0
while index < rounds:
    temp = [stock_datas[i][index + 30][0] for i in range(len(stock_datas))]
    target.append(temp)
    index += 1
    
y = np.asarray(target)
y_train = y[0: train_index].astype('float32')
y_test = y[train_index: rounds].astype('float32')

print(y_train.shape, y_test.shape)

(1784, 37) (199, 37)


In [36]:
epochs = 100

model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = epochs, shuffle = True, batch_size = 100, verbose= 1)

Train on 1784 samples, validate on 199 samples
Epoch 1/100
1784/1784 [==============================] - 153s 86ms/step - loss: 0.0010 - val_loss: 2.2197e-04
Epoch 2/100
1784/1784 [==============================] - 1s 479us/step - loss: 9.8192e-04 - val_loss: 2.1927e-04
Epoch 3/100
1784/1784 [==============================] - 1s 472us/step - loss: 9.4808e-04 - val_loss: 2.1904e-04
Epoch 4/100
1784/1784 [==============================] - 1s 470us/step - loss: 9.4111e-04 - val_loss: 2.1917e-04
Epoch 5/100
1784/1784 [==============================] - 1s 479us/step - loss: 9.4111e-04 - val_loss: 2.1914e-04
Epoch 6/100
1784/1784 [==============================] - 1s 474us/step - loss: 9.3340e-04 - val_loss: 2.1923e-04
Epoch 7/100
1784/1784 [==============================] - 1s 480us/step - loss: 9.3247e-04 - val_loss: 2.1929e-04
Epoch 8/100
1784/1784 [==============================] - 1s 476us/step - loss: 9.2366e-04 - val_loss: 2.1936e-04
Epoch 9/100
1784/1784 [=============================

1784/1784 [==============================] - 1s 475us/step - loss: 8.5453e-04 - val_loss: 2.1848e-04
Epoch 73/100
1784/1784 [==============================] - 1s 473us/step - loss: 8.5318e-04 - val_loss: 2.1848e-04
Epoch 74/100
1784/1784 [==============================] - 1s 473us/step - loss: 8.5312e-04 - val_loss: 2.1850e-04
Epoch 75/100
1784/1784 [==============================] - 1s 470us/step - loss: 8.5104e-04 - val_loss: 2.1851e-04
Epoch 76/100
1784/1784 [==============================] - 1s 465us/step - loss: 8.4787e-04 - val_loss: 2.1850e-04
Epoch 77/100
1784/1784 [==============================] - 1s 475us/step - loss: 8.4652e-04 - val_loss: 2.1845e-04
Epoch 78/100
1784/1784 [==============================] - 1s 473us/step - loss: 8.4848e-04 - val_loss: 2.1843e-04
Epoch 79/100
1784/1784 [==============================] - 1s 479us/step - loss: 8.4473e-04 - val_loss: 2.1843e-04
Epoch 80/100
1784/1784 [==============================] - 1s 470us/step - loss: 8.5506e-04 - val_loss

In [255]:
predict = model.predict(x_test, verbose = 0)

print(predict)

[[  4.43944242e-03]
 [  1.56791846e-03]
 [  1.23050064e-04]
 [  1.06722268e-03]
 [  1.43525749e-03]
 [  1.35611917e-03]
 [  1.19774602e-03]
 [  8.66791059e-04]
 [  1.23048550e-03]
 [  6.91056601e-04]
 [ -6.86041312e-06]
 [  1.62228348e-03]]


In [256]:
mse = mean_square_error(classes, y_test)
print(mse)

6.41176494518e-06
